<a href="https://colab.research.google.com/github/arthursl12/POC1/blob/main/POC2_FD001_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
%pip install scikeras;
%pip install -U tensorflow-addons;

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.1 MB 5.0 MB/s 


In [2]:
import os
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import glob
import datetime

In [3]:
from sklearn.pipeline import Pipeline
from sklearn.compose import TransformedTargetRegressor

from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [4]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout

from tensorflow.keras.metrics import RootMeanSquaredError as RMSE
from tensorflow_addons.metrics import RSquare as R2

import tensorflow_addons as tfa

from scikeras.wrappers import KerasRegressor

In [5]:
sns.set_palette('colorblind')

In [6]:
# Reproducibility
seed = 42
os.environ['PYTHONHASHSEED']=str(seed)
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [7]:
tf.config.experimental.enable_op_determinism()

# Data Preparation

In [8]:
# Dataset Download 
os.system('git clone https://github.com/arthursl12/dataset_2')
os.system('mv /content/dataset_2/CMaps /content/CMaps')
os.system('mv /content/dataset_2/data_processing /content/data_processing')
os.system('rm -rf dataset_2')

0

In [9]:
from data_processing.processing import DatasetProcessing
from data_processing.training import HyperparameterSearch, reclipper_scorer
from data_processing.eval import Evaluation

In [10]:
proc = DatasetProcessing()

## Data Integration

The data are provided as a zip-compressed text file with 26 columns of numbers, separated by spaces. Each row is a snapshot of data taken during a single operational cycle, each column is a different variable. The columns correspond to:  

1) unit number   
2) time, in cycles  
3) operational setting 1  
4) operational setting 2  
5) operational setting 3    
6) sensor measurement 1    
7) sensor measurement 2  
...  
26) sensor measurement 20


There are 6 conditions (or combinations) which the 3 operational settings can take.  
Condition 1: Altitude = 0, Mach Number = 0, TRA = 100  
Condition 2: Altitude = 10, Mach Number = 0.25, TRA = 100  
Condition 3: Altitude = 20, Mach Number = 0.7 TRA = 100  
Condition 4: Altitude = 25, Mach Number = 0.62, TRA = 60  
Condition 5: Altitude = 35 Mach Number = 0.84, TRA = 100  
Condition 6: Altitude = 42, Mach Number = 0.84, TRA = 100  
  
There is slight variation in all these conditions so you may get numbers like 24.453 instead of 25 exactly.

FD001: Condition 1 only  
FD002: Mix of all the conditions  
FD003: Condition 1 only  
FD004: Mix of all conditions  


In [11]:
index_cols, settings_cols, sensors_cols, cols = proc.column_names()
train, test, y_test = proc.read_dataset(1)
train

,unit_number,time,op_1,op_2,op_3,s_0,s_1,s_2,s_3,s_4,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,1,3,-0.0043,0.0003,100.0,518.67,642.35,1587.99,1404.20,14.62,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,1,4,0.0007,0.0000,100.0,518.67,642.35,1582.79,1401.87,14.62,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,1,5,-0.0019,-0.0002,100.0,518.67,642.37,1582.85,1406.22,14.62,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20626,100,196,-0.0004,-0.0003,100.0,518.67,643.49,1597.98,1428.63,14.62,...,519.49,2388.26,8137.60,8.4956,0.03,397,2388,100.0,38.49,22.9735
20627,100,197,-0.0016,-0.0005,100.0,518.67,643.54,1604.50,1433.58,14.62,...,519.68,2388.22,8136.50,8.5139,0.03,395,2388,100.0,38.30,23.1594
20628,100,198,0.0004,0.0000,100.0,518.67,643.42,1602.46,1428.18,14.62,...,520.01,2388.24,8141.05,8.5646,0.03,398,2388,100.0,38.44,22.9333
20629,100,199,-0.0011,0.0003,100.0,518.67,643.23,1605.26,1426.53,14.62,...,519.67,2388.23,8139.29,8.5389,0.03,395,2388,100.0,38.29,23.0640


## Preprocessing

### Validation Set

In [12]:
from sklearn.model_selection import GroupShuffleSplit

def train_val_split(train):
    gss = GroupShuffleSplit(n_splits=1, train_size=0.80, random_state=42)  
    for idx_train, idx_val in gss.split(train,groups=train["unit_number"]):
        # print('train_split_engines', train.iloc[idx_train]['unit_number'].unique(), '\n')
        # print('validate_split_engines', train.iloc[idx_val]['unit_number'].unique(), '\n')

        df_train = train.iloc[idx_train].copy()
        df_val = train.iloc[idx_val].copy()

    return df_train, df_val

In [13]:
df_train, df_val = train_val_split(train)

In [14]:
X_t_train, y_t_train = proc.X_y_train_divide(df_train)
X_t_val, y_t_val = proc.X_y_train_divide(df_val)

### Test Set Transformation 
Test set has samples for all cycles, but has annotations only for last one

In [15]:
test.shape, y_test.shape

((13096, 26), (100, 1))

In [16]:
test_last = proc.transform_test(test)
test_last.head()

,s_0,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,s_9,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,518.67,642.58,1581.22,1398.91,14.62,21.61,554.42,2388.08,9056.40,1.3,...,521.79,2388.06,8130.11,8.4024,0.03,393,2388,100.0,38.81,23.3552
1,518.67,642.55,1586.59,1410.83,14.62,21.61,553.52,2388.10,9044.77,1.3,...,521.74,2388.09,8126.90,8.4505,0.03,391,2388,100.0,38.81,23.2618
2,518.67,642.88,1589.75,1418.89,14.62,21.61,552.59,2388.16,9049.26,1.3,...,520.83,2388.14,8131.46,8.4119,0.03,395,2388,100.0,38.93,23.2740
3,518.67,642.78,1594.53,1406.88,14.62,21.61,552.64,2388.13,9051.30,1.3,...,521.88,2388.11,8133.64,8.4634,0.03,395,2388,100.0,38.58,23.2581
4,518.67,642.27,1589.94,1419.36,14.62,21.61,553.29,2388.10,9053.99,1.3,...,521.00,2388.15,8125.74,8.4362,0.03,394,2388,100.0,38.75,23.4117


In [17]:
X_test = test_last

### Remaining Useful Life (RUL)

In [18]:
train = proc.add_remaining_useful_life_linear(train)
train[index_cols+['RUL']].head()

,unit_number,time,RUL
0,1,1,191
1,1,2,190
2,1,3,189
3,1,4,188
4,1,5,187


## Attributes and target separation

In [19]:
X_train, y_train = proc.X_y_train_divide(train)

In [20]:
y_train.head()

,RUL
0,191
1,190
2,189
3,188
4,187


In [21]:
X_train.head()

,s_0,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,s_9,...,s_11,s_12,s_13,s_14,s_15,s_16,s_17,s_18,s_19,s_20
0,518.67,641.82,1589.70,1400.60,14.62,21.61,554.36,2388.06,9046.19,1.3,...,521.66,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190
1,518.67,642.15,1591.82,1403.14,14.62,21.61,553.75,2388.04,9044.07,1.3,...,522.28,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236
2,518.67,642.35,1587.99,1404.20,14.62,21.61,554.26,2388.08,9052.94,1.3,...,522.42,2388.03,8133.23,8.4178,0.03,390,2388,100.0,38.95,23.3442
3,518.67,642.35,1582.79,1401.87,14.62,21.61,554.45,2388.11,9049.48,1.3,...,522.86,2388.08,8133.83,8.3682,0.03,392,2388,100.0,38.88,23.3739
4,518.67,642.37,1582.85,1406.22,14.62,21.61,554.00,2388.06,9055.15,1.3,...,522.19,2388.04,8133.80,8.4294,0.03,393,2388,100.0,38.90,23.4044


## Training and Evaluation functions

In [22]:
eval = Evaluation()

In [23]:
search = HyperparameterSearch()

# MLP

## Linear RUL

In [24]:
from tensorflow.keras.optimizers import SGD, Adam, RMSprop
from tensorflow.keras.optimizers.schedules import ExponentialDecay

In [34]:
def create_model(optim=Adam, layer_nodes=[16,32,64], 
                 dropout=0.1, activation="relu", learning_rate=1e-4, 
                 loss='mean_squared_error',metrics=[tf.keras.metrics.MeanSquaredError()]):
    train_cols = sensors_cols
    input_dim = len(train_cols)

    model = Sequential()
    model.add(Dense(layer_nodes[0], input_dim=input_dim, activation=activation))
    model.add(Dropout(dropout))
    model.add(Dense(layer_nodes[1], activation=activation))
    model.add(Dropout(dropout))
    model.add(Dense(layer_nodes[2], activation=activation))
    model.add(Dropout(dropout))
    model.add(Dense(1))
    model.compile(loss=loss, optimizer=optim(learning_rate=learning_rate), metrics=metrics)

    return model
reg = KerasRegressor(model=create_model)

In [26]:
# es = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)
es = tf.keras.callbacks.EarlyStopping(monitor='loss', 
                                      patience=5, restore_best_weights=True)

In [27]:
from keras.callbacks import LambdaCallback

def printLog(epoch, logs):
    print(
        f"E {epoch}\t: loss={logs['loss']:.3f}, "+
        f"rmse={logs['root_mean_squared_error']:.3f}, "+
        f"r2={logs['r_square']:.3f}; "+
        f"v_loss={logs['val_loss']:.3f}, "+
        f"v_rmse={logs['val_root_mean_squared_error']:.3f}, "+
        f"v_r2={logs['val_r_square']:.3f}; "
    )

printerCallback = LambdaCallback(on_epoch_end=printLog)

In [28]:
ED1 = ExponentialDecay(initial_learning_rate=1e-2, 
                       decay_steps=100000, decay_rate=0.96)
ED2 = ExponentialDecay(initial_learning_rate=1e-2, 
                       decay_steps=100000, decay_rate=0.8)
ED3 = ExponentialDecay(initial_learning_rate=1e-1, 
                       decay_steps=100000, decay_rate=0.96)

In [ ]:
model = Pipeline([
    ('scaler'  ,   MinMaxScaler()),
    ('mlp'  ,   KerasRegressor(model=create_model,verbose=0, callbacks=[es], 
                               validation_split=0.2, 
                               model__metrics=[RMSE(), R2()],
                               model__loss='mse'))
])
model

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('mlp',
                 KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7f7877599a90>], model=<function create_model at 0x7f7817703560>, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f781771f9d0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f781771ad10>], validation_split=0.2, verbose=0))])

In [ ]:
# ~4min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [MinMaxScaler(),StandardScaler()],
        "mlp__epochs": [1,5,10,20,30,50],
        "mlp__validation_split":[0.1,0.2,0.4,0.5],
        "mlp__batch_size": [32,64,128,256,512],
        "mlp__model__optim":[Adam,SGD,RMSprop],
        "mlp__model__learning_rate": [1e-2,1e-3,1e-4],
        "mlp__model__dropout": [0.1,0.2,0.4,0.5],
        "mlp__model__layer_nodes": [[16, 32, 64], [32, 64, 128], [64, 128, 256], [128, 256, 512]],
        "mlp__model__activation": ["relu","tanh", "sigmoid"]
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                            scorer='r2', ignore_warnings=True, cv=2)
    print(model)
    print("Finished:", datetime.datetime.now())

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 2 folds for each of 41 candidates, totalling 82 fits


----------
iter: 1
n_candidates: 14
n_resources: 1500
Fitting 2 folds for each of 14 candidates, totalling 28 fits
----------
iter: 2
n_candidates: 5
n_resources: 4500
Fitting 2 folds for each of 5 candidates, totalling 10 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 2 folds for each of 2 candidates, totalling 4 fits
Best params:  {'scaler': StandardScaler(), 'mlp__validation_split': 0.2, 'mlp__model__optim': <class 'keras.optimizer_v2.gradient_descent.SGD'>, 'mlp__model__learning_rate': 0.0001, 'mlp__model__layer_nodes': [128, 256, 512], 'mlp__model__dropout': 0.1, 'mlp__model__activation': 'relu', 'mlp__epochs': 30, 'mlp__batch_size': 512}
Pipeline(steps=[('scaler', StandardScaler()),
                ('mlp',
                 KerasRegressor(batch_size=512, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f776d481dd0>], epochs=30, model=<function create_model at 0x7f7817703560>, model__activation='relu', model__dropout=0.1, model__layer_nodes=[128, 256, 512]

Best Model:
```
{'scaler': StandardScaler(), 
'mlp__validation_split': 0.2, 
'mlp__model__optim': <class 'keras.optimizer_v2.gradient_descent.SGD'>, 'mlp__model__learning_rate': 0.0001, 
'mlp__model__layer_nodes': [128, 256, 512], 
'mlp__model__dropout': 0.1, 
'mlp__model__activation': 'relu', 
'mlp__epochs': 30, 
'mlp__batch_size': 512}
```

In [ ]:
model = Pipeline([
    ('scaler'  , StandardScaler()),
    ('mlp'     , KerasRegressor(model=create_model, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                verbose=0, callbacks=[es,printerCallback],  
                                validation_split=0.2, 
                                model__optim=SGD,
                                model__learning_rate=0.0001, 
                                model__layer_nodes=[128,256,512],
                                model__dropout=0.1,
                                model__activation="relu",
                                epochs=30, 
                                batch_size=512))
])
model

Pipeline(steps=[('scaler', StandardScaler()),
                ('mlp',
                 KerasRegressor(batch_size=512, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f7877599a90>, <keras.callbacks.LambdaCallback object at 0x7f781771fc50>], epochs=30, model=<function create_model at 0x7f7817703560>, model__activation='relu', model__dropout=0.1, model__layer_nodes=[128, 256, 512], model__learning_rate=0.0001, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f779a430090>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f7877599d90>], model__optim=<class 'keras.optimizer_v2.gradient_descent.SGD'>, validation_split=0.2, verbose=0))])

In [ ]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
print("Finished:", datetime.datetime.now())

E 0	: loss=10880.258, rmse=104.308, r2=-1.503; v_loss=4040.997, v_rmse=63.569, v_r2=0.347; 
E 1	: loss=2077.079, rmse=45.575, r2=0.522; v_loss=2666.208, v_rmse=51.635, v_r2=0.569; 
E 2	: loss=1706.493, rmse=41.310, r2=0.607; v_loss=2602.992, v_rmse=51.020, v_r2=0.579; 
E 3	: loss=1653.722, rmse=40.666, r2=0.620; v_loss=2544.440, v_rmse=50.442, v_r2=0.589; 
E 4	: loss=1633.723, rmse=40.419, r2=0.624; v_loss=2536.447, v_rmse=50.363, v_r2=0.590; 
E 5	: loss=1616.827, rmse=40.210, r2=0.628; v_loss=2539.150, v_rmse=50.390, v_r2=0.590; 
E 6	: loss=1627.588, rmse=40.343, r2=0.626; v_loss=2441.406, v_rmse=49.411, v_r2=0.605; 
E 7	: loss=1621.175, rmse=40.264, r2=0.627; v_loss=2576.533, v_rmse=50.760, v_r2=0.584; 
E 8	: loss=1600.923, rmse=40.012, r2=0.632; v_loss=2522.857, v_rmse=50.228, v_r2=0.592; 
E 9	: loss=1610.057, rmse=40.126, r2=0.630; v_loss=2904.245, v_rmse=53.891, v_r2=0.531; 
E 10	: loss=1605.266, rmse=40.066, r2=0.631; v_loss=2689.315, v_rmse=51.859, v_r2=0.565; 
E 11	: loss=1599.

In [ ]:
eval.show_result(y_test, model.predict(X_test))
print("Finished:", datetime.datetime.now())

R2=0.440,RMSE=-31.100
Finished: 2022-09-11 16:26:05.503277


## Non-linear RUL

In [ ]:
model = Pipeline([
    ('scaler'   ,   MinMaxScaler()),
    ('trf_reg'  ,   TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KerasRegressor(
                                model=create_model,verbose=0, callbacks=[es], 
                                validation_split=0.2, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse'),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':50})))
])
model

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7f7877599a90>], model=<function create_model at 0x7f7817703560>, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f7773fd7490>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f7772799d50>], validation_split=0.2, verbose=0),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f78932e34d0>,
                                                                            kw_args={'a_max': 50,
                                                                                     'a_min': 0})))])

In [ ]:
# ~10min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [MinMaxScaler(),StandardScaler()],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1),
        "trf_reg__regressor__epochs": [1,5,10,20,30,50],
        "trf_reg__regressor__validation_split":[0.1,0.2,0.4,0.5],
        "trf_reg__regressor__batch_size": [32,64,128,256,512],
        "trf_reg__regressor__model__optim":[Adam,RMSprop],
        "trf_reg__regressor__model__learning_rate": [1e-2,1e-3,1e-4],
        "trf_reg__regressor__model__dropout": [0.1,0.2,0.4,0.5],
        "trf_reg__regressor__model__layer_nodes": [[16, 32, 64], [32, 64, 128], [64, 128, 256], [128, 256, 512]],
        "trf_reg__regressor__model__activation": ["relu","tanh", "sigmoid"]
    }

    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                            scorer='r2', ignore_warnings=True, cv=2)
    print(model)
    print("Finished:", datetime.datetime.now())

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 2 folds for each of 41 candidates, totalling 82 fits
----------
iter: 1
n_candidates: 14
n_resources: 1500
Fitting 2 folds for each of 14 candidates, totalling 28 fits
----------
iter: 2
n_candidates: 5
n_resources: 4500
Fitting 2 folds for each of 5 candidates, totalling 10 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 2 folds for each of 2 candidates, totalling 4 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 133}, 'trf_reg__regressor__validation_split': 0.1, 'trf_reg__regressor__model__optim': <class 'keras.optimizer_v2.adam.Adam'>, 'trf_reg__regressor__model__learning_rate': 0.001, 'trf_reg__regressor__model__layer_nodes': [64, 128, 256], 'trf_reg__regressor__model__dropout': 0.1, 'trf_reg__regressor__model__activation': 'relu',

Best Model:
```

{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 133},   
'trf_reg__regressor__validation_split': 0.1,   
'trf_reg__regressor__model__optim': <class 'keras.optimizer_v2.adam.Adam'>, 
'trf_reg__regressor__model__learning_rate': 0.001, 
'trf_reg__regressor__model__layer_nodes': [64, 128, 256],
'trf_reg__regressor__model__dropout': 0.1, 
'trf_reg__regressor__model__activation': 'relu', 
'trf_reg__regressor__epochs': 10, 
'trf_reg__regressor__batch_size': 64, 
'scaler': StandardScaler()}
```

In [ ]:
model = Pipeline([
    ('scaler'   ,   MinMaxScaler()),
    ('trf_reg'  ,   TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KerasRegressor(
                                model=create_model, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                verbose=0, callbacks=[es,printerCallback],  
                                validation_split=0.1, 
                                model__optim=Adam,
                                model__learning_rate=0.001, 
                                model__layer_nodes=[64, 128, 256],
                                model__dropout=0.1,
                                model__activation="relu",
                                epochs=10, 
                                batch_size=64),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':133})))
])
model

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=KerasRegressor(batch_size=64, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f7877599a90>, <keras.callbacks.LambdaCallback object at 0x7f781771fc50>], epochs=10, model=<function create_model at 0x7f7817703560>, model__activation='relu', model__dro...1, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f772a14e410>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f772a160c50>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, validation_split=0.1, verbose=0),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f78932e34d0>,
                                                                            kw_args={'a_max': 133,
                                                                            

In [ ]:
# model.fit(X_train, y_train)
# reclipped_y =  model['trf_reg'].transformer.transform(y_train)
# eval.show_result(reclipped_y, model.predict(X_train))
# eval.show_result_cv(reclipped_y, X_train, model)

model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
print("Finished:", datetime.datetime.now())
# eval.show_result_cv(y_train, X_train, model)

E 0	: loss=1983.879, rmse=44.541, r2=-0.002; v_loss=511.479, v_rmse=22.616, v_r2=0.737; 
E 1	: loss=561.978, rmse=23.706, r2=0.716; v_loss=458.431, v_rmse=21.411, v_r2=0.765; 
E 2	: loss=521.000, rmse=22.825, r2=0.737; v_loss=456.708, v_rmse=21.371, v_r2=0.766; 
E 3	: loss=494.589, rmse=22.239, r2=0.750; v_loss=436.360, v_rmse=20.889, v_r2=0.776; 
E 4	: loss=478.880, rmse=21.883, r2=0.758; v_loss=431.351, v_rmse=20.769, v_r2=0.779; 
E 5	: loss=464.849, rmse=21.560, r2=0.765; v_loss=411.838, v_rmse=20.294, v_r2=0.789; 
E 6	: loss=463.246, rmse=21.523, r2=0.766; v_loss=402.338, v_rmse=20.058, v_r2=0.794; 
E 7	: loss=459.012, rmse=21.425, r2=0.768; v_loss=471.187, v_rmse=21.707, v_r2=0.758; 
E 8	: loss=442.389, rmse=21.033, r2=0.777; v_loss=420.845, v_rmse=20.515, v_r2=0.784; 
E 9	: loss=440.591, rmse=20.990, r2=0.777; v_loss=395.998, v_rmse=19.900, v_r2=0.797; 
R2=0.799,RMSE=-19.946
Finished: 2022-09-11 16:56:44.219760


In [ ]:
# reclipped_y =  model['trf_reg'].transformer.transform(y_test)
# eval.show_result(reclipped_y, model.predict(X_test))

In [ ]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))
print("Finished:", datetime.datetime.now())

R2=0.770,RMSE=-19.671
Finished: 2022-09-11 16:56:49.493359


The best result we had so far in training and in testing

## PolyFeatures(2) + Linear RUL

In [39]:
P2size = PolynomialFeatures().fit_transform(X_train).shape
P2size

(20631, 253)

In [40]:
train_cols = sensors_cols
len(train_cols)

21

In [49]:
def create_model_pol(optim=Adam, layer_nodes=[16,32,64], 
                 dropout=0.1, activation="relu", learning_rate=1e-4, 
                 loss='mean_squared_error',metrics=[tf.keras.metrics.MeanSquaredError()]):
    train_cols = sensors_cols
    input_dim = len(train_cols)
    P2size = PolynomialFeatures().fit_transform(X_train).shape[1]

    model = Sequential()
    model.add(Dense(layer_nodes[0], input_dim=P2size, activation=activation))
    model.add(Dropout(dropout))
    model.add(Dense(layer_nodes[1], activation=activation))
    model.add(Dropout(dropout))
    model.add(Dense(layer_nodes[2], activation=activation))
    model.add(Dropout(dropout))
    model.add(Dense(1))
    model.compile(loss=loss, optimizer=optim(learning_rate=learning_rate), metrics=metrics)

    return model
reg = KerasRegressor(model=create_model)

In [44]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('scaler'  ,   MinMaxScaler()),
    ('mlp'  ,   KerasRegressor(model=create_model_pol,verbose=0, callbacks=[es], 
                               validation_split=0.2, 
                               model__metrics=[RMSE(), R2()],
                               model__loss='mse'))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures()), ('scaler', MinMaxScaler()),
                ('mlp',
                 KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7f36c0d21d10>], model=<function create_model at 0x7f36b04c3320>, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f362c2ea850>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f36b0528250>], validation_split=0.2, verbose=0))])

In [45]:
# ~6min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [MinMaxScaler(),StandardScaler()],
        "mlp__epochs": [1,5,10,20,30,50],
        "mlp__validation_split":[0.1,0.2,0.4,0.5],
        "mlp__batch_size": [32,64,128,256,512],
        "mlp__model__optim":[Adam,SGD,RMSprop],
        "mlp__model__learning_rate": [1e-2,1e-3,1e-4],
        "mlp__model__dropout": [0.1,0.2,0.4,0.5],
        "mlp__model__layer_nodes": [[16, 32, 64], [32, 64, 128], [64, 128, 256], [128, 256, 512]],
        "mlp__model__activation": ["relu","tanh", "sigmoid"]
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                            scorer='r2', ignore_warnings=True, cv=2)
    print(model)
    print("Finished:", datetime.datetime.now())

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 2 folds for each of 41 candidates, totalling 82 fits


----------
iter: 1
n_candidates: 14
n_resources: 1500
Fitting 2 folds for each of 14 candidates, totalling 28 fits
----------
iter: 2
n_candidates: 5
n_resources: 4500
Fitting 2 folds for each of 5 candidates, totalling 10 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 2 folds for each of 2 candidates, totalling 4 fits
Best params:  {'scaler': StandardScaler(), 'mlp__validation_split': 0.2, 'mlp__model__optim': <class 'keras.optimizer_v2.gradient_descent.SGD'>, 'mlp__model__learning_rate': 0.001, 'mlp__model__layer_nodes': [128, 256, 512], 'mlp__model__dropout': 0.4, 'mlp__model__activation': 'sigmoid', 'mlp__epochs': 50, 'mlp__batch_size': 512}
Pipeline(steps=[('poly_ft', PolynomialFeatures()), ('scaler', StandardScaler()),
                ('mlp',
                 KerasRegressor(batch_size=512, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f36281d8550>], epochs=50, model=<function create_model at 0x7f36b04c3320>, model__activation='sigmoid', model__dropout

Best Model:
```
{'scaler': StandardScaler(), 
'mlp__validation_split': 0.2, 
'mlp__model__optim': <class 'keras.optimizer_v2.gradient_descent.SGD'>, 'mlp__model__learning_rate': 0.001, 
'mlp__model__layer_nodes': [128, 256, 512], 
'mlp__model__dropout': 0.4, 
'mlp__model__activation': 'sigmoid', 
'mlp__epochs': 50, 
'mlp__batch_size': 512}
```

In [46]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('scaler'  , StandardScaler()),
    ('mlp'     , KerasRegressor(model=create_model_pol, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                verbose=0, callbacks=[es,printerCallback],  
                                validation_split=0.2, 
                                model__optim=SGD,
                                model__learning_rate=0.001, 
                                model__layer_nodes=[128,256,512],
                                model__dropout=0.4,
                                model__activation='sigmoid',
                                epochs=50, 
                                batch_size=512))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures()), ('scaler', StandardScaler()),
                ('mlp',
                 KerasRegressor(batch_size=512, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f36c0d21d10>, <keras.callbacks.LambdaCallback object at 0x7f36c01c4850>], epochs=50, model=<function create_model at 0x7f36b04c3320>, model__activation='sigmoid', model__dropout=0.4, model__layer_nodes=[128, 256, 512], model__learning_rate=0.001, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f361648bd50>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f361648b850>], model__optim=<class 'keras.optimizer_v2.gradient_descent.SGD'>, validation_split=0.2, verbose=0))])

In [47]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
print("Finished:", datetime.datetime.now())

E 0	: loss=4271.573, rmse=65.357, r2=0.017; v_loss=3707.898, v_rmse=60.893, v_r2=0.401; 
E 1	: loss=1938.243, rmse=44.025, r2=0.554; v_loss=2839.112, v_rmse=53.283, v_r2=0.541; 
E 2	: loss=1759.316, rmse=41.944, r2=0.595; v_loss=2711.072, v_rmse=52.068, v_r2=0.562; 
E 3	: loss=1734.720, rmse=41.650, r2=0.601; v_loss=2709.544, v_rmse=52.053, v_r2=0.562; 
E 4	: loss=1716.355, rmse=41.429, r2=0.605; v_loss=2621.782, v_rmse=51.203, v_r2=0.576; 
E 5	: loss=1714.445, rmse=41.406, r2=0.606; v_loss=2598.654, v_rmse=50.977, v_r2=0.580; 
E 6	: loss=1699.979, rmse=41.231, r2=0.609; v_loss=2570.351, v_rmse=50.699, v_r2=0.585; 
E 7	: loss=1699.774, rmse=41.228, r2=0.609; v_loss=2664.598, v_rmse=51.620, v_r2=0.569; 
E 8	: loss=1669.670, rmse=40.862, r2=0.616; v_loss=2607.967, v_rmse=51.068, v_r2=0.579; 
E 9	: loss=1662.341, rmse=40.772, r2=0.618; v_loss=2865.036, v_rmse=53.526, v_r2=0.537; 
E 10	: loss=1645.317, rmse=40.563, r2=0.622; v_loss=2657.781, v_rmse=51.554, v_r2=0.570; 
E 11	: loss=1641.543

In [48]:
eval.show_result(y_test, model.predict(X_test))
print("Finished:", datetime.datetime.now())

R2=0.469,RMSE=-30.277
Finished: 2022-09-12 10:50:06.977277


## PolyFeatures(3) + Linear RUL

In [67]:
def create_model_pol3(optim=Adam, layer_nodes=[16,32,64], 
                 dropout=0.1, activation="relu", learning_rate=1e-4, 
                 loss='mean_squared_error',metrics=[tf.keras.metrics.MeanSquaredError()]):
    train_cols = sensors_cols
    input_dim = len(train_cols)
    P2size = PolynomialFeatures(degree=3).fit_transform(X_train).shape[1]

    model = Sequential()
    model.add(Dense(layer_nodes[0], input_dim=P2size, activation=activation))
    model.add(Dropout(dropout))
    model.add(Dense(layer_nodes[1], activation=activation))
    model.add(Dropout(dropout))
    model.add(Dense(layer_nodes[2], activation=activation))
    model.add(Dropout(dropout))
    model.add(Dense(1))
    model.compile(loss=loss, optimizer=optim(learning_rate=learning_rate), metrics=metrics)

    return model
reg = KerasRegressor(model=create_model)

In [68]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=3)),
    ('scaler'  ,   MinMaxScaler()),
    ('mlp'  ,   KerasRegressor(model=create_model_pol3,verbose=0, callbacks=[es], 
                               validation_split=0.2, 
                               model__metrics=[RMSE(), R2()],
                               model__loss='mse'))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3)),
                ('scaler', MinMaxScaler()),
                ('mlp',
                 KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7f36c0d21d10>], model=<function create_model_pol3 at 0x7f361b2a1dd0>, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f361b402f10>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f361b402090>], validation_split=0.2, verbose=0))])

In [69]:
# ~6min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [MinMaxScaler(),StandardScaler()],
        "mlp__epochs": [1,5,10,20,30,50],
        "mlp__validation_split":[0.1,0.2,0.4,0.5],
        "mlp__batch_size": [32,64,128,256,512],
        "mlp__model__optim":[Adam,SGD,RMSprop],
        "mlp__model__learning_rate": [1e-2,1e-3,1e-4],
        "mlp__model__dropout": [0.1,0.2,0.4,0.5],
        "mlp__model__layer_nodes": [[16, 32, 64], [32, 64, 128], [64, 128, 256], [128, 256, 512]],
        "mlp__model__activation": ["relu","tanh", "sigmoid"]
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                            scorer='r2', ignore_warnings=True, cv=2)
    print(model)
    print("Finished:", datetime.datetime.now())

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 2 folds for each of 41 candidates, totalling 82 fits
----------
iter: 1
n_candidates: 14
n_resources: 1500
Fitting 2 folds for each of 14 candidates, totalling 28 fits
----------
iter: 2
n_candidates: 5
n_resources: 4500
Fitting 2 folds for each of 5 candidates, totalling 10 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 2 folds for each of 2 candidates, totalling 4 fits
Best params:  {'scaler': StandardScaler(), 'mlp__validation_split': 0.2, 'mlp__model__optim': <class 'keras.optimizer_v2.gradient_descent.SGD'>, 'mlp__model__learning_rate': 0.001, 'mlp__model__layer_nodes': [128, 256, 512], 'mlp__model__dropout': 0.4, 'mlp__model__activation': 'sigmoid', 'mlp__epochs': 50, 'mlp__batch_size': 512}
Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3)),
     

Best Model:
```
{'scaler': StandardScaler(), 
'mlp__validation_split': 0.2, 
'mlp__model__optim': <class 'keras.optimizer_v2.gradient_descent.SGD'>,
'mlp__model__learning_rate': 0.001, 
'mlp__model__layer_nodes': [128, 256, 512], 
'mlp__model__dropout': 0.4, 
'mlp__model__activation': 'sigmoid', 
'mlp__epochs': 50, 
'mlp__batch_size': 512}
```

In [70]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=3)),
    ('scaler'  , StandardScaler()),
    ('mlp'     , KerasRegressor(model=create_model_pol3, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                verbose=0, callbacks=[es,printerCallback],  
                                validation_split=0.2, 
                                model__optim=SGD,
                                model__learning_rate=0.001, 
                                model__layer_nodes=[128,256,512],
                                model__dropout=0.4,
                                model__activation='sigmoid',
                                epochs=50, 
                                batch_size=512))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3)),
                ('scaler', StandardScaler()),
                ('mlp',
                 KerasRegressor(batch_size=512, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f36c0d21d10>, <keras.callbacks.LambdaCallback object at 0x7f36c01c4850>], epochs=50, model=<function create_model_pol3 at 0x7f361b2a1dd0>, model__activation='sigmoid', model__dropout=0.4, model__layer_nodes=[128, 256, 512], model__learning_rate=0.001, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f3628307ad0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f3628307a50>], model__optim=<class 'keras.optimizer_v2.gradient_descent.SGD'>, validation_split=0.2, verbose=0))])

In [71]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
print("Finished:", datetime.datetime.now())

E 0	: loss=3859.454, rmse=62.125, r2=0.112; v_loss=3244.542, v_rmse=56.961, v_r2=0.476; 
E 1	: loss=1869.120, rmse=43.233, r2=0.570; v_loss=3304.891, v_rmse=57.488, v_r2=0.466; 
E 2	: loss=1816.944, rmse=42.626, r2=0.582; v_loss=2708.521, v_rmse=52.043, v_r2=0.562; 
E 3	: loss=1755.046, rmse=41.893, r2=0.596; v_loss=2778.461, v_rmse=52.711, v_r2=0.551; 
E 4	: loss=1729.353, rmse=41.585, r2=0.602; v_loss=2693.361, v_rmse=51.898, v_r2=0.565; 
E 5	: loss=1723.992, rmse=41.521, r2=0.603; v_loss=2670.318, v_rmse=51.675, v_r2=0.568; 
E 6	: loss=1698.269, rmse=41.210, r2=0.609; v_loss=2668.869, v_rmse=51.661, v_r2=0.569; 
E 7	: loss=1697.971, rmse=41.206, r2=0.609; v_loss=2667.741, v_rmse=51.650, v_r2=0.569; 
E 8	: loss=1675.807, rmse=40.937, r2=0.615; v_loss=2689.505, v_rmse=51.860, v_r2=0.565; 
E 9	: loss=1668.688, rmse=40.850, r2=0.616; v_loss=3041.413, v_rmse=55.149, v_r2=0.508; 
E 10	: loss=1650.078, rmse=40.621, r2=0.620; v_loss=2714.094, v_rmse=52.097, v_r2=0.561; 
E 11	: loss=1645.906

In [72]:
eval.show_result(y_test, model.predict(X_test))
print("Finished:", datetime.datetime.now())

R2=0.401,RMSE=-32.164
Finished: 2022-09-12 11:33:45.020869


Mesmo modelo do Poly2

## PolyFeatures(2) + Non-Linear RUL

In [50]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('scaler'   ,   MinMaxScaler()),
    ('trf_reg'  ,   TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KerasRegressor(
                                model=create_model_pol,verbose=0, callbacks=[es], 
                                validation_split=0.2, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse'),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':50})))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures()), ('scaler', MinMaxScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7f36c0d21d10>], model=<function create_model_pol at 0x7f36162f0cb0>, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f36286c33d0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f36286c3b50>], validation_split=0.2, verbose=0),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f373c87e4d0>,
                                                                            kw_args={'a_max': 50,
                                                                                     'a_min': 0})))])

In [51]:
# ~10min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [MinMaxScaler(),StandardScaler()],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1),
        "trf_reg__regressor__epochs": [1,5,10,20,30,50],
        "trf_reg__regressor__validation_split":[0.1,0.2,0.4,0.5],
        "trf_reg__regressor__batch_size": [32,64,128,256,512],
        "trf_reg__regressor__model__optim":[Adam,RMSprop],
        "trf_reg__regressor__model__learning_rate": [1e-2,1e-3,1e-4],
        "trf_reg__regressor__model__dropout": [0.1,0.2,0.4,0.5],
        "trf_reg__regressor__model__layer_nodes": [[16, 32, 64], [32, 64, 128], [64, 128, 256], [128, 256, 512]],
        "trf_reg__regressor__model__activation": ["relu","tanh", "sigmoid"]
    }

    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                            scorer='r2', ignore_warnings=True, cv=2)
    print(model)
    print("Finished:", datetime.datetime.now())

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 2 folds for each of 41 candidates, totalling 82 fits
----------
iter: 1
n_candidates: 14
n_resources: 1500
Fitting 2 folds for each of 14 candidates, totalling 28 fits
----------
iter: 2
n_candidates: 5
n_resources: 4500
Fitting 2 folds for each of 5 candidates, totalling 10 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 2 folds for each of 2 candidates, totalling 4 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 135}, 'trf_reg__regressor__validation_split': 0.5, 'trf_reg__regressor__model__optim': <class 'keras.optimizer_v2.adam.Adam'>, 'trf_reg__regressor__model__learning_rate': 0.01, 'trf_reg__regressor__model__layer_nodes': [64, 128, 256], 'trf_reg__regressor__model__dropout': 0.5, 'trf_reg__regressor__model__activation': 'tanh', 

Best Model:
```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 135}, 
'trf_reg__regressor__validation_split': 0.5, 
'trf_reg__regressor__model__optim': <class 'keras.optimizer_v2.adam.Adam'>, 
'trf_reg__regressor__model__learning_rate': 0.01, 
'trf_reg__regressor__model__layer_nodes': [64, 128, 256], 
'trf_reg__regressor__model__dropout': 0.5, 
'trf_reg__regressor__model__activation': 'tanh', 
'trf_reg__regressor__epochs': 30, 
'trf_reg__regressor__batch_size': 256, 
'scaler': StandardScaler()}
```

In [54]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('scaler'   ,   StandardScaler()),
    ('trf_reg'  ,   TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KerasRegressor(
                                model=create_model_pol, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                verbose=0, callbacks=[es,printerCallback],  
                                validation_split=0.5, 
                                model__optim=Adam,
                                model__learning_rate=0.01, 
                                model__layer_nodes=[64, 128, 256],
                                model__dropout=0.5,
                                model__activation="tanh",
                                epochs=30, 
                                batch_size=256),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':135})))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures()), ('scaler', StandardScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=KerasRegressor(batch_size=256, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f36c0d21d10>, <keras.callbacks.LambdaCallback object at 0x7f36c01c4850>], epochs=30, model=<function create_model_pol at 0x7f36162f0...1, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f36165ef7d0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f36165ef690>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, validation_split=0.5, verbose=0),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f373c87e4d0>,
                                                                            kw_args={'a_max': 135,
                                                                            

In [55]:
# model.fit(X_train, y_train)
# reclipped_y =  model['trf_reg'].transformer.transform(y_train)
# eval.show_result(reclipped_y, model.predict(X_train))
# eval.show_result_cv(reclipped_y, X_train, model)

model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
print("Finished:", datetime.datetime.now())
# eval.show_result_cv(y_train, X_train, model)

E 0	: loss=5314.098, rmse=72.898, r2=-1.617; v_loss=1288.184, v_rmse=35.891, v_r2=0.370; 
E 1	: loss=812.940, rmse=28.512, r2=0.600; v_loss=665.227, v_rmse=25.792, v_r2=0.675; 
E 2	: loss=576.975, rmse=24.020, r2=0.716; v_loss=567.214, v_rmse=23.816, v_r2=0.722; 
E 3	: loss=526.989, rmse=22.956, r2=0.741; v_loss=540.345, v_rmse=23.245, v_r2=0.736; 
E 4	: loss=509.114, rmse=22.564, r2=0.749; v_loss=530.718, v_rmse=23.037, v_r2=0.740; 
E 5	: loss=509.515, rmse=22.572, r2=0.749; v_loss=527.402, v_rmse=22.965, v_r2=0.742; 
E 6	: loss=506.129, rmse=22.497, r2=0.751; v_loss=529.495, v_rmse=23.011, v_r2=0.741; 
E 7	: loss=498.015, rmse=22.316, r2=0.755; v_loss=560.912, v_rmse=23.684, v_r2=0.726; 
E 8	: loss=495.849, rmse=22.268, r2=0.756; v_loss=550.318, v_rmse=23.459, v_r2=0.731; 
E 9	: loss=479.884, rmse=21.906, r2=0.764; v_loss=544.229, v_rmse=23.329, v_r2=0.734; 
E 10	: loss=492.787, rmse=22.199, r2=0.757; v_loss=545.064, v_rmse=23.347, v_r2=0.733; 
E 11	: loss=492.276, rmse=22.187, r2=0.

In [56]:
# reclipped_y =  model['trf_reg'].transformer.transform(y_test)
# eval.show_result(reclipped_y, model.predict(X_test))

In [57]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))
print("Finished:", datetime.datetime.now())

R2=0.753,RMSE=-20.475
Finished: 2022-09-12 11:06:20.553177


## PolyFeatures(3) + Non-Linear RUL

In [60]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=3)),
    ('scaler'   ,   MinMaxScaler()),
    ('trf_reg'  ,   TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KerasRegressor(
                                model=create_model_pol3,verbose=0, callbacks=[es], 
                                validation_split=0.2, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse'),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':50})))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3)),
                ('scaler', MinMaxScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7f36c0d21d10>], model=<function create_model_pol3 at 0x7f361d4e1ef0>, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f361d5f16d0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f36167e2750>], validation_split=0.2, verbose=0),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f373c87e4d0>,
                                                                            kw_args={'a_max': 50,
                                                                                     'a_min': 0})))])

In [61]:
# ~10min
GRID_SEARCH = False
if (GRID_SEARCH):
    param_distributions = {
        "scaler": [MinMaxScaler(),StandardScaler()],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1),
        "trf_reg__regressor__epochs": [1,5,10,20,30,50],
        "trf_reg__regressor__validation_split":[0.1,0.2,0.4,0.5],
        "trf_reg__regressor__batch_size": [32,64,128,256,512],
        "trf_reg__regressor__model__optim":[Adam,RMSprop],
        "trf_reg__regressor__model__learning_rate": [1e-2,1e-3,1e-4],
        "trf_reg__regressor__model__dropout": [0.1,0.2,0.4,0.5],
        "trf_reg__regressor__model__layer_nodes": [[16, 32, 64], [32, 64, 128], [64, 128, 256], [128, 256, 512]],
        "trf_reg__regressor__model__activation": ["relu","tanh", "sigmoid"]
    }

    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                            scorer='r2', ignore_warnings=True, cv=2)
    print(model)
    print("Finished:", datetime.datetime.now())

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 2 folds for each of 41 candidates, totalling 82 fits
----------
iter: 1
n_candidates: 14
n_resources: 1500
Fitting 2 folds for each of 14 candidates, totalling 28 fits
----------
iter: 2
n_candidates: 5
n_resources: 4500
Fitting 2 folds for each of 5 candidates, totalling 10 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 2 folds for each of 2 candidates, totalling 4 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 133}, 'trf_reg__regressor__validation_split': 0.1, 'trf_reg__regressor__model__optim': <class 'keras.optimizer_v2.adam.Adam'>, 'trf_reg__regressor__model__learning_rate': 0.001, 'trf_reg__regressor__model__layer_nodes': [64, 128, 256], 'trf_reg__regressor__model__dropout': 0.1, 'trf_reg__regressor__model__activation': 'relu',

Best Model:
```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 133}, 
'trf_reg__regressor__validation_split': 0.1, 
'trf_reg__regressor__model__optim': <class 'keras.optimizer_v2.adam.Adam'>, 
'trf_reg__regressor__model__learning_rate': 0.001, 
'trf_reg__regressor__model__layer_nodes': [64, 128, 256], 
'trf_reg__regressor__model__dropout': 0.1, 
'trf_reg__regressor__model__activation': 'relu', 
'trf_reg__regressor__epochs': 30,
 'trf_reg__regressor__batch_size': 64, 
 'scaler': StandardScaler()}
```

In [64]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=3)),
    ('scaler'   ,   StandardScaler()),
    ('trf_reg'  ,   TransformedTargetRegressor(
        check_inverse=False,
        regressor   = KerasRegressor(
                                model=create_model_pol3, 
                                model__metrics=[RMSE(), R2()],
                                model__loss='mse',
                                verbose=0, callbacks=[es,printerCallback],  
                                validation_split=0.1, 
                                model__optim=Adam,
                                model__learning_rate=0.001, 
                                model__layer_nodes=[64, 128, 256],
                                model__dropout=0.1,
                                model__activation="relu",
                                epochs=30, 
                                batch_size=64),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':133})))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3)),
                ('scaler', StandardScaler()),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=KerasRegressor(batch_size=64, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f36c0d21d10>, <keras.callbacks.LambdaCallback object at 0x7f36c01c4850>], epochs=30, model=<function create_model_pol3 at 0x7...1, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f3628705c90>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f3642112b50>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, validation_split=0.1, verbose=0),
                                            transformer=FunctionTransformer(func=<function clip at 0x7f373c87e4d0>,
                                                                            kw_args={'a_max': 133,
                                                            

In [65]:
# model.fit(X_train, y_train)
# reclipped_y =  model['trf_reg'].transformer.transform(y_train)
# eval.show_result(reclipped_y, model.predict(X_train))
# eval.show_result_cv(reclipped_y, X_train, model)

model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
print("Finished:", datetime.datetime.now())
# eval.show_result_cv(y_train, X_train, model)

E 0	: loss=1465.280, rmse=38.279, r2=0.260; v_loss=877.040, v_rmse=29.615, v_r2=0.550; 
E 1	: loss=758.275, rmse=27.537, r2=0.617; v_loss=639.657, v_rmse=25.291, v_r2=0.672; 
E 2	: loss=628.628, rmse=25.072, r2=0.682; v_loss=544.522, v_rmse=23.335, v_r2=0.721; 
E 3	: loss=556.142, rmse=23.583, r2=0.719; v_loss=592.383, v_rmse=24.339, v_r2=0.696; 
E 4	: loss=532.007, rmse=23.065, r2=0.731; v_loss=452.094, v_rmse=21.263, v_r2=0.768; 
E 5	: loss=501.482, rmse=22.394, r2=0.747; v_loss=453.017, v_rmse=21.284, v_r2=0.767; 
E 6	: loss=502.090, rmse=22.407, r2=0.746; v_loss=424.875, v_rmse=20.612, v_r2=0.782; 
E 7	: loss=489.451, rmse=22.124, r2=0.753; v_loss=482.244, v_rmse=21.960, v_r2=0.752; 
E 8	: loss=482.443, rmse=21.965, r2=0.756; v_loss=482.506, v_rmse=21.966, v_r2=0.752; 
E 9	: loss=481.390, rmse=21.941, r2=0.757; v_loss=438.913, v_rmse=20.950, v_r2=0.775; 
E 10	: loss=481.096, rmse=21.934, r2=0.757; v_loss=506.945, v_rmse=22.515, v_r2=0.740; 
E 11	: loss=480.237, rmse=21.914, r2=0.75

In [ ]:
# reclipped_y =  model['trf_reg'].transformer.transform(y_test)
# eval.show_result(reclipped_y, model.predict(X_test))

In [66]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))
print("Finished:", datetime.datetime.now())

R2=0.811,RMSE=-17.814
Finished: 2022-09-12 11:21:51.443498


# OFF

In [ ]:
model = Pipeline([
    ('scaler'  ,   MinMaxScaler()),
    ('mlp'  ,   KerasRegressor(model=create_model,verbose=0, callbacks=[es], validation_split=0.2))
])
model

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('mlp',
                 KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7fdf39694cd0>], model=<function create_model at 0x7fdf49de63b0>, validation_split=0.2, verbose=0))])

In [ ]:
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV, HalvingGridSearchCV


search = HalvingRandomSearchCV(model, param_distributions,
                               min_resources=500, scoring='r2', 
                               random_state=42, verbose=2, cv=3)


search.fit(X_train, y_train);
print("Best params: ", search.best_params_)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 3 folds for each of 41 candidates, totalling 123 fits
[CV] END mlp__batch_size=128, mlp__epochs=50, mlp__model__activation=tanh, mlp__model__dropout=0.5, mlp__model__layer_nodes=[64, 128, 256], mlp__model__learning_rate=<keras.optimizer_v2.learning_rate_schedule.ExponentialDecay object at 0x7fd0a7c35d50>, mlp__model__optim=<class 'keras.optimizer_v2.gradient_descent.SGD'>, mlp__validation_split=0.5, scaler=MinMaxScaler(); total time=   0.8s
[CV] END mlp__batch_size=128, mlp__epochs=50, mlp__model__activation=tanh, mlp__model__dropout=0.5, mlp__model__layer_nodes=[64, 128, 256], mlp__model__learning_rate=<keras.optimizer_v2.learning_rate_schedule.ExponentialDecay object at 0x7fd0a7c35d50>, mlp__model__optim=<class 'keras.optimizer_v2.gradient_descent.SGD'>, mlp__validation_split=0.

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 221, in __call__
    sample_weight=sample_weight,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 790, in r2_score
    y_true, y_pred, multioutput
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 96, in _check_reg_targets
    y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)


[CV] END mlp__batch_size=256, mlp__epochs=20, mlp__model__activation=relu, mlp__model__dropout=0.4, mlp__model__layer_nodes=[128, 256, 512], mlp__model__learning_rate=<keras.optimizer_v2.learning_rate_schedule.ExponentialDecay object at 0x7fd0a7c35d50>, mlp__model__optim=<class 'keras.optimizer_v2.gradient_descent.SGD'>, mlp__validation_split=0.2, scaler=StandardScaler(); total time=   0.8s


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 221, in __call__
    sample_weight=sample_weight,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 790, in r2_score
    y_true, y_pred, multioutput
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 96, in _check_reg_targets
    y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)


[CV] END mlp__batch_size=256, mlp__epochs=20, mlp__model__activation=relu, mlp__model__dropout=0.4, mlp__model__layer_nodes=[128, 256, 512], mlp__model__learning_rate=<keras.optimizer_v2.learning_rate_schedule.ExponentialDecay object at 0x7fd0a7c35d50>, mlp__model__optim=<class 'keras.optimizer_v2.gradient_descent.SGD'>, mlp__validation_split=0.2, scaler=StandardScaler(); total time=   0.8s


...

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 221, in __call__
    sample_weight=sample_weight,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 790, in r2_score
    y_true, y_pred, multioutput
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 96, in _check_reg_targets
    y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)


[CV] END mlp__batch_size=32, mlp__epochs=10, mlp__model__activation=relu, mlp__model__dropout=0.5, mlp__model__layer_nodes=[128, 256, 512], mlp__model__learning_rate=0.001, mlp__model__optim=<class 'keras.optimizer_v2.gradient_descent.SGD'>, mlp__validation_split=0.5, scaler=StandardScaler(); total time=   0.9s


/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py:774: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 221, in __call__
    sample_weight=sample_weight,
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 790, in r2_score
    y_true, y_pred, multioutput
  File "/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_regression.py", line 96, in _check_reg_targets
    y_pred = check_array(y_pred, ensure_2d=False, dtype=dtype)


[CV] END mlp__batch_size=32, mlp__epochs=10, mlp__model__activation=relu, mlp__model__dropout=0.5, mlp__model__layer_nodes=[128, 256, 512], mlp__model__learning_rate=0.001, mlp__model__optim=<class 'keras.optimizer_v2.gradient_descent.SGD'>, mlp__validation_split=0.5, scaler=StandardScaler(); total time=   0.9s
[CV] END mlp__batch_size=32, mlp__epochs=50, mlp__model__activation=sigmoid, mlp__model__dropout=0.5, mlp__model__layer_nodes=[32, 64, 128], mlp__model__learning_rate=0.0001, mlp__model__optim=<class 'keras.optimizer_v2.adam.Adam'>, mlp__validation_split=0.2, scaler=MinMaxScaler(); total time=   3.2s
[CV] END mlp__batch_size=32, mlp__epochs=50, mlp__model__activation=sigmoid, mlp__model__dropout=0.5, mlp__model__layer_nodes=[32, 64, 128], mlp__model__learning_rate=0.0001, mlp__model__optim=<class 'keras.optimizer_v2.adam.Adam'>, mlp__validation_split=0.2, scaler=MinMaxScaler(); total time=   3.1s
[CV] END mlp__batch_size=32, mlp__epochs=50, mlp__model__activation=sigmoid, mlp__m

KeyboardInterrupt: ignored

In [ ]:
model = Pipeline([
    ('scaler'  , MinMaxScaler()),
    ('mlp'     , KerasRegressor(model=create_model, verbose=0, callbacks=[es,printerCallback], 
                                batch_size=128, model__learning_rate=0.01,
                                validation_split=0.5, epochs=20, 
                                model__optim=Adam, model__layer_nodes=[16,32,64],
                                model__dropout=0.1, model__activation='relu', 
                                model__metrics=[tf.keras.metrics.RootMeanSquaredError(), tfa.metrics.RSquare()],
                                model__loss='mse'))
])
model

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('mlp',
                 KerasRegressor(batch_size=128, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f64ae3187d0>, <keras.callbacks.LambdaCallback object at 0x7f64aebf8b10>], epochs=20, model=<function create_model at 0x7f6542c000e0>, model__activation='relu', model__dropout=0.1, model__layer_nodes=[16, 32, 64], model__learning_rate=0.01, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f64ae30f6d0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f64ae317350>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, validation_split=0.5, verbose=0))])

In [ ]:
model.fit(X_train, y_train)

E 0	: loss=5207.414, rmse=72.162, r2=-0.345; v_loss=3045.556, v_rmse=55.187, v_r2=0.451; 
E 1	: loss=1527.117, rmse=39.078, r2=0.606; v_loss=2813.525, v_rmse=53.043, v_r2=0.493; 
E 2	: loss=1359.615, rmse=36.873, r2=0.649; v_loss=2666.109, v_rmse=51.634, v_r2=0.519; 
E 3	: loss=1315.130, rmse=36.265, r2=0.660; v_loss=2877.303, v_rmse=53.640, v_r2=0.481; 
E 4	: loss=1290.427, rmse=35.923, r2=0.667; v_loss=2632.022, v_rmse=51.303, v_r2=0.526; 
E 5	: loss=1294.368, rmse=35.977, r2=0.666; v_loss=2702.958, v_rmse=51.990, v_r2=0.513; 
E 6	: loss=1239.708, rmse=35.209, r2=0.680; v_loss=3236.657, v_rmse=56.892, v_r2=0.417; 
E 7	: loss=1272.573, rmse=35.673, r2=0.671; v_loss=3314.008, v_rmse=57.567, v_r2=0.403; 
E 8	: loss=1270.919, rmse=35.650, r2=0.672; v_loss=2804.315, v_rmse=52.956, v_r2=0.494; 
E 9	: loss=1257.236, rmse=35.458, r2=0.675; v_loss=2682.475, v_rmse=51.793, v_r2=0.516; 
E 10	: loss=1252.234, rmse=35.387, r2=0.677; v_loss=2580.146, v_rmse=50.795, v_r2=0.535; 
E 11	: loss=1235.93

Pipeline(steps=[('scaler', MinMaxScaler()),
                ('mlp',
                 KerasRegressor(batch_size=128, callbacks=[<keras.callbacks.EarlyStopping object at 0x7f64ae3187d0>, <keras.callbacks.LambdaCallback object at 0x7f64aebf8b10>], epochs=20, model=<function create_model at 0x7f6542c000e0>, model__activation='relu', model__dropout=0.1, model__layer_nodes=[16, 32, 64], model__learning_rate=0.01, model__loss='mse', model__metrics=[<keras.metrics.RootMeanSquaredError object at 0x7f64ae30f6d0>, <tensorflow_addons.metrics.r_square.RSquare object at 0x7f64ae317350>], model__optim=<class 'keras.optimizer_v2.adam.Adam'>, validation_split=0.5, verbose=0))])

In [ ]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
# eval.show_result_cv(y_train, X_train, model)

E 0	: loss=4787.087, rmse=61.810, r2=0.195; v_loss=3012.669, v_rmse=54.888, v_r2=0.457; 
E 1	: loss=1573.484, rmse=39.667, r2=0.594; v_loss=2781.886, v_rmse=52.744, v_r2=0.499; 
E 2	: loss=1422.959, rmse=37.722, r2=0.632; v_loss=2648.300, v_rmse=51.462, v_r2=0.523; 
E 3	: loss=1322.943, rmse=36.372, r2=0.658; v_loss=2887.253, v_rmse=53.733, v_r2=0.480; 
E 4	: loss=1293.984, rmse=35.972, r2=0.666; v_loss=2671.713, v_rmse=51.689, v_r2=0.518; 
E 5	: loss=1264.718, rmse=35.563, r2=0.673; v_loss=2667.296, v_rmse=51.646, v_r2=0.519; 
E 6	: loss=1209.704, rmse=34.781, r2=0.688; v_loss=3272.676, v_rmse=57.207, v_r2=0.410; 
E 7	: loss=1235.243, rmse=35.146, r2=0.681; v_loss=2919.450, v_rmse=54.032, v_r2=0.474; 
E 8	: loss=1193.297, rmse=34.544, r2=0.692; v_loss=2588.395, v_rmse=50.876, v_r2=0.533; 
E 9	: loss=1183.923, rmse=34.408, r2=0.694; v_loss=2595.500, v_rmse=50.946, v_r2=0.532; 
E 10	: loss=1186.067, rmse=34.439, r2=0.694; v_loss=2638.092, v_rmse=51.362, v_r2=0.524; 
E 11	: loss=1173.344

In [ ]:
eval.show_result(y_test, model.predict(X_test))

R2=0.643,RMSE=-24.825


In [ ]:
dir(search)

['__abstractmethods__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_abc_impl',
 '_check_feature_names',
 '_check_input_parameters',
 '_check_n_features',
 '_check_refit_for_multimetric',
 '_checked_cv_orig',
 '_estimator_type',
 '_format_results',
 '_generate_candidate_params',
 '_get_param_names',
 '_get_tags',
 '_more_tags',
 '_n_samples_orig',
 '_pairwise',
 '_repr_html_',
 '_repr_html_inner',
 '_repr_mimebundle_',
 '_required_parameters',
 '_run_search',
 '_select_best_index',
 '_validate_data',
 'aggressive_elimination',
 'best_estimator_',
 'best_index_',
 'best_params_',
 'best_score_',
 'classes_',
 'cv',
 'cv_results_',


In [ ]:
search.score

<bound method BaseSearchCV.score of HalvingRandomSearchCV(cv=3,
                      estimator=Pipeline(steps=[('scaler', MinMaxScaler()),
                                                ('mlp',
                                                 KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7fdf39694cd0>], model=<function create_model at 0x7fdf49de63b0>, validation_split=0.2, verbose=0))]),
                      min_resources=500,
                      param_distributions={'mlp__epochs': [1, 5, 10, 20, 30,
                                                           50],
                                           'mlp__model__activation': ['relu'...
                                           'mlp__model__optim': [<class 'keras.optimizer_v2.adam.Adam'>,
                                                                 <class 'keras.optimizer_v2.gradient_descent.SGD'>,
                                                                 <class 'keras.optimizer_v2.rmsprop.R

## Linear RUL

In [ ]:
model = Pipeline([
    ('tree_reg'  ,   DecisionTreeRegressor(random_state=42))
])

In [ ]:
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "tree_reg__criterion": ["squared_error","friedman_mse","absolute_error","poisson"],
        "tree_reg__splitter": ["best", "random"],
        "tree_reg__max_depth": [None,100,200,300],
        "tree_reg__min_samples_split": [2,5,10,15,20],
        "tree_reg__min_samples_leaf": [1,2,5,10,15,20,50,100],
        "tree_reg__max_features": ["sqrt", "log2"],
        "tree_reg__min_impurity_decrease": list(np.arange(0,150)/10),
        "tree_reg__ccp_alpha": list(np.round(np.linspace(0, 2, 81), decimals=3)),
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions, 
                            scorer='r2', ignore_warnings=True)
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 5 folds for each of 41 candidates, totalling 205 fits
----------
iter: 1
n_candidates: 14
n_resources: 1500
Fitting 5 folds for each of 14 candidates, totalling 70 fits
----------
iter: 2
n_candidates: 5
n_resources: 4500
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'tree_reg__splitter': 'best', 'tree_reg__min_samples_split': 2, 'tree_reg__min_samples_leaf': 20, 'tree_reg__min_impurity_decrease': 3.5, 'tree_reg__max_features': 'log2', 'tree_reg__max_depth': 200, 'tree_reg__criterion': 'friedman_mse', 'tree_reg__ccp_alpha': 0.675}
Pipeline(steps=[('tree_reg',
                 DecisionTreeRegressor(ccp_alpha=0.675,
                          

Best Model:
```
('tree_reg',
DecisionTreeRegressor(ccp_alpha=0.675,
                    criterion='friedman_mse', max_depth=200,
                    max_features='log2',
                    min_impurity_decrease=3.5,
                    min_samples_leaf=20, random_state=42))
```

In [ ]:
model = Pipeline([
    ('tree_reg'  ,   DecisionTreeRegressor(ccp_alpha=0.675,
                                    criterion='friedman_mse', max_depth=200,
                                    max_features='log2',
                                    min_impurity_decrease=3.5,
                                    min_samples_leaf=20, random_state=42))
])
model

Pipeline(steps=[('tree_reg',
                 DecisionTreeRegressor(ccp_alpha=0.675,
                                       criterion='friedman_mse', max_depth=200,
                                       max_features='log2',
                                       min_impurity_decrease=3.5,
                                       min_samples_leaf=20, random_state=42))])

In [ ]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=0.645,RMSE=-41.037
(CV) R2=0.537,RMSE=-46.205


In [ ]:
eval.show_result(y_test, model.predict(X_test))

R2=0.298,RMSE=-34.821


Worse than simple linear regression

## Non-linear RUL

In [ ]:
model = Pipeline([
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = DecisionTreeRegressor(random_state=42),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':50})))
    ])

In [ ]:
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(80,150,1),
        "trf_reg__regressor__criterion": ["squared_error","friedman_mse","absolute_error","poisson"],
        "trf_reg__regressor__splitter": ["best", "random"],
        "trf_reg__regressor__max_depth": [None,100,200,300],
        "trf_reg__regressor__min_samples_split": [2,5,10,15,20],
        "trf_reg__regressor__min_samples_leaf": [1,2,5,10,15,20,50,100],
        "trf_reg__regressor__max_features": ["sqrt", "log2"],
        "trf_reg__regressor__min_impurity_decrease": list(np.arange(0,150)/10),
        "trf_reg__regressor__ccp_alpha": list(np.round(np.linspace(0, 2, 81), decimals=3)) 
    
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions)
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 5 folds for each of 41 candidates, totalling 205 fits
----------
iter: 1
n_candidates: 14
n_resources: 1500
Fitting 5 folds for each of 14 candidates, totalling 70 fits
----------
iter: 2
n_candidates: 5
n_resources: 4500
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 98}, 'trf_reg__regressor__splitter': 'best', 'trf_reg__regressor__min_samples_split': 2, 'trf_reg__regressor__min_samples_leaf': 10, 'trf_reg__regressor__min_impurity_decrease': 5.5, 'trf_reg__regressor__max_features': 'sqrt', 'trf_reg__regressor__max_depth': 200, 'trf_reg__regressor__criterion': 'friedman_mse', 'trf_reg__r

Best Model:

```
{'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 98}}

DecisionTreeRegressor(ccp_alpha=0.375,
                    criterion='friedman_mse',
                    max_depth=200,
                    max_features='sqrt',
                    min_impurity_decrease=5.5,
                    min_samples_leaf=10,
                    random_state=42),
```



In [ ]:
model = Pipeline([
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = DecisionTreeRegressor(ccp_alpha=0.375,
                            criterion='friedman_mse',
                            max_depth=200,
                            max_features='sqrt',
                            min_impurity_decrease=5.5,
                            min_samples_leaf=10,
                            random_state=42),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':98})))
    ])
model

Pipeline(steps=[('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=DecisionTreeRegressor(ccp_alpha=0.375,
                                                                            criterion='friedman_mse',
                                                                            max_depth=200,
                                                                            max_features='sqrt',
                                                                            min_impurity_decrease=5.5,
                                                                            min_samples_leaf=10,
                                                                            random_state=42),
                                            transformer=FunctionTransformer(func=<function clip at 0x7fd35e8e7cb0>,
                                                                            kw_args={'a_max': 98,
                

In [ ]:
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

R2=0.815,RMSE=-13.536
(CV) R2=0.781,RMSE=-14.680


In [ ]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))

R2=0.819,RMSE=-14.277


The best result we had so far in training and in testing

## PolyFeatures + Linear RUL

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('tree_reg'  ,   DecisionTreeRegressor(random_state=42))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures()),
                ('tree_reg', DecisionTreeRegressor(random_state=42))])

In [ ]:
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False],
        "tree_reg__criterion": ["squared_error","friedman_mse","absolute_error","poisson"],
        "tree_reg__splitter": ["best", "random"],
        "tree_reg__max_depth": [None,100,200,300],
        "tree_reg__min_samples_split": [2,5,10,15,20],
        "tree_reg__min_samples_leaf": [1,2,5,10,15,20,50,100],
        "tree_reg__max_features": [None, "sqrt", "log2"],
        "tree_reg__min_impurity_decrease": list(np.arange(0,150)/10),
        "tree_reg__ccp_alpha": list(np.round(np.linspace(0, 2, 81), decimals=3)) 
    }
    model = search.run_HR_GS(model, X_train, y_train, 
                            param_distributions, scorer='r2')
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 5 folds for each of 41 candidates, totalling 205 fits
----------
iter: 1
n_candidates: 14
n_resources: 1500
Fitting 5 folds for each of 14 candidates, totalling 70 fits
----------
iter: 2
n_candidates: 5
n_resources: 4500
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'tree_reg__splitter': 'random', 'tree_reg__min_samples_split': 15, 'tree_reg__min_samples_leaf': 15, 'tree_reg__min_impurity_decrease': 6.9, 'tree_reg__max_features': 'sqrt', 'tree_reg__max_depth': 100, 'tree_reg__criterion': 'squared_error', 'tree_reg__ccp_alpha': 1.025, 'poly_ft__interaction_only': False, 'poly_ft__include_bias': True, 'poly_ft__degree': 3}
Pipeline(steps=[('

Best Model:
```
PolynomialFeatures(degree=3)
DecisionTreeRegressor(ccp_alpha=1.025, max_depth=100,
                    max_features='sqrt',
                    min_impurity_decrease=6.9,
                    min_samples_leaf=15,
                    min_samples_split=15, random_state=42,
                    splitter='random'))
```


In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures(degree=3)),
    ('tree_reg'  ,   DecisionTreeRegressor(ccp_alpha=1.025, max_depth=100,
                                        max_features='sqrt',
                                        min_impurity_decrease=6.9,
                                        min_samples_leaf=15,
                                        min_samples_split=15, random_state=42,
                                        splitter='random'))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3)),
                ('tree_reg',
                 DecisionTreeRegressor(ccp_alpha=1.025, max_depth=100,
                                       max_features='sqrt',
                                       min_impurity_decrease=6.9,
                                       min_samples_leaf=15,
                                       min_samples_split=15, random_state=42,
                                       splitter='random'))])

In [ ]:
model.fit(X_train, y_train)
eval.show_result(y_train, model.predict(X_train))
eval.show_result_cv(y_train, X_train, model)

R2=0.580,RMSE=-44.656
(CV) R2=0.550,RMSE=-45.641


In [ ]:
eval.show_result(y_test, model.predict(X_test))

R2=0.311,RMSE=-34.484


No clear benefit from polynomial features in linear RUL, as with the other models. Actually, it worsened the results.

## PolyFeatures + Non-Linear RUL

In [ ]:
model = Pipeline([
    ('poly_ft'  ,   PolynomialFeatures()),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = DecisionTreeRegressor(random_state=42),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':96})))
])

In [ ]:
GRID_SEARCH = True
if (GRID_SEARCH):
    param_distributions = {
        "poly_ft__degree": [1,2,3],
        "poly_ft__interaction_only": [False, True],
        "poly_ft__include_bias": [True, False],
        "trf_reg__transformer__kw_args": search.generate_clip_dicts(70,150,1),
        "trf_reg__regressor__criterion": ["squared_error","friedman_mse","absolute_error","poisson"],
        "trf_reg__regressor__splitter": ["best", "random"],
        "trf_reg__regressor__max_depth": [None,100,200,300],
        "trf_reg__regressor__min_samples_split": [2,5,10,15,20],
        "trf_reg__regressor__min_samples_leaf": [1,2,5,10,15,20,50,100],
        "trf_reg__regressor__max_features": [None, "sqrt", "log2"],
        "trf_reg__regressor__min_impurity_decrease": list(np.arange(0,150)/10),
        "trf_reg__regressor__ccp_alpha": list(np.round(np.linspace(0, 2, 81), decimals=3)) 
    }
    model = search.run_HR_GS(model, X_train, y_train, param_distributions)
    print(model)

n_iterations: 4
n_required_iterations: 4
n_possible_iterations: 4
min_resources_: 500
max_resources_: 20631
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 41
n_resources: 500
Fitting 5 folds for each of 41 candidates, totalling 205 fits
----------
iter: 1
n_candidates: 14
n_resources: 1500
Fitting 5 folds for each of 14 candidates, totalling 70 fits
----------
iter: 2
n_candidates: 5
n_resources: 4500
Fitting 5 folds for each of 5 candidates, totalling 25 fits
----------
iter: 3
n_candidates: 2
n_resources: 13500
Fitting 5 folds for each of 2 candidates, totalling 10 fits
Best params:  {'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 71}, 'trf_reg__regressor__splitter': 'random', 'trf_reg__regressor__min_samples_split': 20, 'trf_reg__regressor__min_samples_leaf': 2, 'trf_reg__regressor__min_impurity_decrease': 8.5, 'trf_reg__regressor__max_features': None, 'trf_reg__regressor__max_depth': 200, 'trf_reg__regressor__criterion': 'squared_error', 'trf_reg__

Best Model:
```
'trf_reg__transformer__kw_args': {'a_min': 0, 'a_max': 71}
PolynomialFeatures(degree=3)
DecisionTreeRegressor(ccp_alpha=1.85,
                        max_depth=200,
                        min_impurity_decrease=8.5,
                        min_samples_leaf=2,
                        min_samples_split=20,
                        random_state=42,
                        splitter='random'),
```


In [ ]:
model = Pipeline([
    ('poly_ft', PolynomialFeatures(degree=3)),
    ('trf_reg' ,TransformedTargetRegressor(
        check_inverse=False,
        regressor   = DecisionTreeRegressor(ccp_alpha=1.85,
                                            max_depth=200,
                                            min_impurity_decrease=8.5,
                                            min_samples_leaf=2,
                                            min_samples_split=20,
                                            random_state=42,
                                            splitter='random'),
        transformer = FunctionTransformer(np.clip, 
                                          kw_args={'a_min':0,'a_max':71})))
])
model

Pipeline(steps=[('poly_ft', PolynomialFeatures(degree=3)),
                ('trf_reg',
                 TransformedTargetRegressor(check_inverse=False,
                                            regressor=DecisionTreeRegressor(ccp_alpha=1.85,
                                                                            max_depth=200,
                                                                            min_impurity_decrease=8.5,
                                                                            min_samples_leaf=2,
                                                                            min_samples_split=20,
                                                                            random_state=42,
                                                                            splitter='random'),
                                            transformer=FunctionTransformer(func=<function clip at 0x7fd35e8e7cb0>,
                                                               

In [ ]:
# ~3min
model.fit(X_train, y_train)
reclipped_y =  model['trf_reg'].transformer.transform(y_train)
eval.show_result(reclipped_y, model.predict(X_train))
eval.show_result_cv(reclipped_y, X_train, model)

R2=0.772,RMSE=-9.983
(CV) R2=0.772,RMSE=-9.954


In [ ]:
reclipped_y =  model['trf_reg'].transformer.transform(y_test)
eval.show_result(reclipped_y, model.predict(X_test))

R2=0.711,RMSE=-12.322


Using the polynominal features actually worsened the perfomance a little bit, when comparing with not using them and the non-linear RUL.